<a href="https://colab.research.google.com/github/LuanMAndrade/Treino-Notebooks-Colab/blob/main/robo_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [ ]:
input_size = 10
rede_neural = Sequential([
    tf.keras.layers.InputLayer(shape=(input_size,)),
    tf.keras.layers.Dense(units=30, activation='relu', kernel_initializer= 'random_uniform' ),
    tf.keras.layers.Dense(units=3, activation='softmax')
])


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
@tf.function
def train_step(state, action_index, reward):
    with tf.GradientTape() as tape:
        q_values = rede_neural(state, training=True)  # shape (1,3)
        q_value = q_values[0, action_index]          # valor da ação escolhida
        loss = tf.square(reward - q_value)           # (r - Q(s,a))²

    grads = tape.gradient(loss, rede_neural.trainable_variables)
    optimizer.apply_gradients(zip(grads, rede_neural.trainable_variables))

In [ ]:
state = np.random.randint(0,101,size=(1,10)) # Gera um estado aleatorio só para teste
possible_actions = [-1, 0, 1] # Define as possiveis ações
output = rede_neural.predict(state) # Toma a decisão, mas sai uma lista de probabilidades
probs = output[0]  # tira o batch dimension
action_index = np.random.choice([0, 1, 2], p=probs) # Escolhe um index baseado nas probabilidades
action = possible_actions[action_index] # Retorna a ação do indice escolhido

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [ ]:
data = [np.random.rand(1, 10).astype(np.float32) for _ in range(1000)] # Gera um dataframe com 1000 estados, cada estado com 10 informações

In [ ]:
memory = []
deslocamento = 0
entradas = []
for indice, i in enumerate(data):
  output = rede_neural.predict(i, verbose=0) # Toma a decisão, mas sai uma lista de probabilidades
  probs = output[0]  # tira o batch dimension
  action_index = np.random.choice([0, 1, 2], p=probs) # Escolhe um index baseado nas probabilidades
  action = possible_actions[action_index] # Retorna a ação do indice escolhido
  memory.append((i[0],action))
  if indice > 500 and action != 0:
    entradas.append(indice)

  if len(memory) > 100:
    del memory[0]
    for indice, valor in enumerate(memory):
      if memory[0][0][0] - memory[indice][0][0] > 5:
        deslocamento = -1
        break
      elif memory[indice][0][0] - memory[0][0][0] > 5:
        deslocamento = 1
        break
      else: deslocamento = 0

  x = action*deslocamento
  if x>0:
    reforco = 1
  elif x<0:
    reforco = -1
  else: reforco = 0

  i_tensor = tf.convert_to_tensor(i, dtype=tf.float32)
  train_step(i_tensor, action_index, reforco)

